1. *Самостоятельно повторить tfidf (документация https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html)

In [1]:
import pandas as pd

Наши новости

In [7]:
news = pd.read_csv("./materials.csv")
print(news.shape)
news.head(3)


(27000, 2)


,doc_id,title
0,6,Заместитель председателяnправительства РФnСерг...
1,4896,Матч 1/16 финала Кубка России по футболу был п...
2,4897,Форвард «Авангарда» Томаш Заборский прокоммент...


Загрузим пользователей и списки последних прочитанных новостей

In [8]:
users = pd.read_csv("users_articles.csv")
users.head(3)

,uid,articles
0,u105138,"[293672, 293328, 293001, 293622, 293126, 1852]"
1,u108690,"[3405, 1739, 2972, 1158, 1599, 322665]"
2,u108339,"[1845, 2009, 2356, 1424, 2939, 323389]"


Итак, нам нужно получить векторные представления пользователей на основе прочитанным ими новостей и самих новостей

### 1. Получаем векторные представления новостей

In [16]:
!pip install razdel pymorphy2
!pip install pymorphy2-dicts

In [17]:
# предобработка текстов
import re
import numpy as np
from gensim.corpora.dictionary import Dictionary
from razdel import tokenize  # сегментация русскоязычного текста на токены и предложения https://github.com/natasha/razdel
import pymorphy2  # Морфологический анализатор

In [18]:
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Andrew\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [19]:
stopword_ru = stopwords.words('russian')
print(len(stopword_ru))

151


In [21]:
with open('./stopwords.txt') as f:
    additional_stopwords = [w.strip() for w in f.readlines() if w]
    
stopword_ru += additional_stopwords
len(stopword_ru)

1401

In [22]:
def clean_text(text):
    '''
    очистка текста
    
    на выходе очищеный текст
    '''
    if not isinstance(text, str):
        text = str(text)
    
    text = text.lower()
    text = text.strip('\n').strip('\r').strip('\t')
    text = re.sub("-\s\r\n\|-\s\r\n|\r\n", '', str(text))

    text = re.sub("[0-9]|[-—.,:;_%©«»?*!@#№$^•·&()]|[+=]|[[]|[]]|[/]|", '', text)
    text = re.sub(r"\r\n\t|\n|\\s|\r\t|\\n", ' ', text)
    text = re.sub(r'[\xad]|[\s+]', ' ', text.strip())
    text = re.sub('n', ' ', text)
    
    return text

cache = {}
morph = pymorphy2.MorphAnalyzer()

def lemmatization(text):    
    '''
    лемматизация
        [0] если зашел тип не `str` делаем его `str`
        [1] токенизация предложения через razdel
        [2] проверка есть ли в начале слова '-'
        [3] проверка токена с одного символа
        [4] проверка есть ли данное слово в кэше
        [5] лемматизация слова
        [6] проверка на стоп-слова

    на выходе лист лемматизированых токенов
    '''

    # [0]
    if not isinstance(text, str):
        text = str(text)
    
    # [1]
    tokens = list(tokenize(text))
    words = [_.text for _ in tokens]

    words_lem = []
    for w in words:
        if w[0] == '-': # [2]
            w = w[1:]
        if len(w) > 1: # [3]
            if w in cache: # [4]
                words_lem.append(cache[w])
            else: # [5]
                temp_cach = cache[w] = morph.parse(w)[0].normal_form
                words_lem.append(temp_cach)
    
    words_lem_without_stopwords = [i for i in words_lem if not i in stopword_ru] # [6]
    
    return words_lem_without_stopwords

In [23]:
from tqdm import tqdm
tqdm.pandas()

# Запускаем очистку текста. Будет долго...
news['title'] = news['title'].progress_apply(lambda x: clean_text(x))

  0%|                                                | 0/27000 [00:00<?, ?it/s]C:\Users\Andrew\AppData\Local\Temp\ipykernel_5332\4083466619.py:14: FutureWarning: Possible nested set at position 39
  text = re.sub("[0-9]|[-—.,:;_%©«»?*!@#№$^•·&()]|[+=]|[[]|[]]|[/]|", '', text)
100%|███████████████████████████████████| 27000/27000 [00:31<00:00, 858.71it/s]


In [24]:
news['title'].iloc[:10]

0    заместитель председателя правительства рф серг...
1    матч  финала кубка россии по футболу был приос...
2    форвард авангарда томаш заборский прокомментир...
3    главный тренер кубани юрий красножан прокоммен...
4    решением попечительского совета владивостокско...
5    ио главного тренера вячеслав буцаев прокоммент...
6    запорожский металлург дома потерпел разгромное...
7    сборная сша одержала победу над австрией со сч...
8    бывший защитник сборной россии дарюс каспарайт...
9    полузащитник цска зоран тошич после победы над...
Name: title, dtype: object

In [25]:
# Запускаем лемматизацию текста. Будет очень долго...
news['title'] = news['title'].progress_apply(lambda x: lemmatization(x))

100%|████████████████████████████████████| 27000/27000 [06:00<00:00, 74.80it/s]


А теперь в 3 строчки обучим нашу модель

In [26]:
# сформируем список наших текстов
texts = list(news['title'].values)

# Создадим корпус из списка с текстами
common_dictionary = Dictionary(texts)
common_corpus = [common_dictionary.doc2bow(text) for text in texts]

Запускаем обучение

In [27]:
N_topic = 20  # можно было менять

In [28]:
from gensim.models import LdaModel

# Обучаем модель на корпусе
lda = LdaModel(common_corpus, num_topics=N_topic, id2word=common_dictionary, passes=2)  # можно было менять

In [29]:
from gensim.test.utils import datapath

# Сохраняем модель на диск
temp_file = datapath("model.lda")
lda.save(temp_file)

In [30]:
# Загружаем обученную модель с диска
lda = LdaModel.load(temp_file)

In [31]:
# Создаем новый корпус документов, которые раньше не видели
other_texts = list(news['title'].iloc[:3])
other_corpus = [common_dictionary.doc2bow(text) for text in other_texts]

unseen_doc = other_corpus[2]
print(other_texts[2])
lda[unseen_doc] 

['форвард', 'авангард', 'томаш', 'заборский', 'прокомментировать', 'игра', 'свой', 'команда', 'матч', 'чемпионат', 'кхл', 'против', 'атланта', 'провести', 'плохой', 'матч', 'нижний', 'новгород', 'против', 'торпедо', 'настраиваться', 'первый', 'минута', 'включиться', 'работа', 'сказать', 'заборский', 'получиться', 'забросить', 'быстрый', 'гол', 'задать', 'хороший', 'темп', 'поединок', 'мочь', 'играть', 'ещё', 'хороший', 'сторона', 'пять', 'очко', 'выезд', 'девять', 'это', 'хороший']


[(1, 0.15785119),
 (6, 0.27091855),
 (7, 0.01724319),
 (10, 0.024811707),
 (11, 0.5123571)]

Обучили модель. Теперь 2 вопроса:

1. как выглядят наши темы
2. как получить для документа вектор значений (вероятности принадлежности каждой теме)

In [32]:
x = lda.show_topics(num_topics=N_topic, num_words=7, formatted=False)
topics_words = [(tp[0], [wd[0] for wd in tp[1]]) for tp in x]

# Печатаем только слова
for topic, words in topics_words:
    print(f"topic_{topic}: " + " ".join(words))

topic_0: исследование станция экономика пациент операция запустить лечение
topic_1: это который год всё мочь весь человек
topic_2: банк фонд взрыв экономический глава россия министр
topic_3: ракета год продукция проверка решение который суд
topic_4: вицепремьер белый индия снять сосед малое пол
topic_5: риск исследование высота белоруссия километр расчёт смерть
topic_6: журнал человек который турция погибнуть турецкий город
topic_7: тур финляндия терминал дания аргентина порт король
topic_8: наука год работа это человек новый который
topic_9: это который президент свой мочь заявить россия
topic_10: рак польша определение вуз польский мэй поляк
topic_11: температура космос команда ночью игра лекарство день
topic_12: япония озеро корея су таиланд золото южный
topic_13: дело год суд ребёнок который сотрудник орган
topic_14: сша украина военный гражданин американский год украинский
topic_15: год который человек время также это стать
topic_16: год университет академия смерть звезда конкурс 

Очень неплохо - большинство тем вполне можно описать о чем они

Давайте напишем функцию, которая будет нам возвращать векторное представление новости

In [33]:
def get_lda_vector(lda, text):
    unseen_doc = common_dictionary.doc2bow(text)
    lda_tuple = lda[unseen_doc]

    not_null_topics = dict(zip([i[0] for i in lda_tuple], [i[1] for i in lda_tuple]))

    output_vector = []
    for i in range(N_topic):
        if i not in not_null_topics:
            output_vector.append(0)
        else:
            output_vector.append(not_null_topics[i])
    return np.array(output_vector)

In [34]:
get_lda_vector(lda, news['title'].iloc[0])

array([0.        , 0.        , 0.79603052, 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.09623289, 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.07466652, 0.01637141, 0.        ])

In [35]:
topic_matrix = pd.DataFrame([get_lda_vector(lda, text) for text in news['title'].values])
topic_matrix.columns = [f'topic_{i}' for i in range(N_topic)]
topic_matrix['doc_id'] = news['doc_id'].values
topic_matrix = topic_matrix[['doc_id']+[f'topic_{i}' for i in range(N_topic)]]
topic_matrix.head(5)

,doc_id,topic_0,topic_1,topic_2,topic_3,topic_4,topic_5,topic_6,topic_7,topic_8,...,topic_10,topic_11,topic_12,topic_13,topic_14,topic_15,topic_16,topic_17,topic_18,topic_19
0,6,0.0,0.000000,0.796051,0.0,0.0,0.000000,0.000000,0.000000,0.0,...,0.000000,0.096233,0.0,0.0,0.000000,0.0,0.000000,0.074641,0.016376,0.000000
1,4896,0.0,0.000000,0.573219,0.0,0.0,0.000000,0.000000,0.000000,0.0,...,0.000000,0.404268,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000
2,4897,0.0,0.157937,0.000000,0.0,0.0,0.000000,0.270983,0.017241,0.0,...,0.024812,0.512209,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000
3,4898,0.0,0.440535,0.000000,0.0,0.0,0.024705,0.000000,0.000000,0.0,...,0.000000,0.212117,0.0,0.0,0.162008,0.0,0.126930,0.000000,0.000000,0.025249
4,4899,0.0,0.000000,0.312295,0.0,0.0,0.000000,0.000000,0.000000,0.0,...,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.662694,0.000000,0.000000,0.000000


Прекрасно, мы получили вектора наших новостей! И даже умеем интерпретировать получившиеся темы.

Можно двигаться далее

### Следующий шаг - векторные представления пользователей

In [36]:
users.head(3)

,uid,articles
0,u105138,"[293672, 293328, 293001, 293622, 293126, 1852]"
1,u108690,"[3405, 1739, 2972, 1158, 1599, 322665]"
2,u108339,"[1845, 2009, 2356, 1424, 2939, 323389]"


In [37]:
doc_dict = dict(zip(topic_matrix['doc_id'].values, topic_matrix[[f'topic_{i}' for i in range(N_topic)]].values))

In [38]:
doc_dict[293672]

array([0.        , 0.        , 0.        , 0.        , 0.        ,
       0.02575488, 0.        , 0.        , 0.        , 0.16390933,
       0.05044567, 0.        , 0.        , 0.        , 0.        ,
       0.37760425, 0.09250495, 0.1299898 , 0.09885481, 0.04759575])

In [39]:
def get_user_embedding(user_articles_list, doc_dict, func):
    user_articles_list = eval(user_articles_list)
    user_vector = np.array([doc_dict[doc_id] for doc_id in user_articles_list])
    user_vector = func(user_vector, axis=0)
    return user_vector

In [40]:
user_articles_list = users['articles'].iloc[33]

get_user_embedding(user_articles_list, doc_dict, np.mean)

array([0.00316257, 0.07301907, 0.0339698 , 0.05248131, 0.        ,
       0.01321948, 0.        , 0.        , 0.06270328, 0.26959733,
       0.01147959, 0.00796858, 0.        , 0.15498565, 0.07931524,
       0.02986641, 0.00256568, 0.12291378, 0.03928091, 0.03116639])

Теперь получим эмбединги для всех пользователей и проверим их качество на конкретной downstream-задаче

In [41]:
FUNC = np.mean

In [42]:
user_embeddings = pd.DataFrame([i for i in users['articles'].apply(lambda x: get_user_embedding(x, doc_dict, FUNC))])
user_embeddings.columns = [f'topic_{i}' for i in range(N_topic)]
user_embeddings['uid'] = users['uid'].values
user_embeddings = user_embeddings[['uid']+[f'topic_{i}' for i in range(N_topic)]]
user_embeddings.head(3)

,uid,topic_0,topic_1,topic_2,topic_3,topic_4,topic_5,topic_6,topic_7,topic_8,...,topic_10,topic_11,topic_12,topic_13,topic_14,topic_15,topic_16,topic_17,topic_18,topic_19
0,u105138,0.030297,0.063619,0.083210,0.007537,0.000000,0.010949,0.031341,0.000000,0.042386,...,0.018451,0.006087,0.0,0.088453,0.041973,0.247529,0.050127,0.037741,0.042514,0.040250
1,u108690,0.007978,0.161570,0.049986,0.038571,0.003282,0.010306,0.025159,0.000000,0.001911,...,0.000000,0.011973,0.0,0.099249,0.123617,0.097997,0.021764,0.048871,0.017788,0.042099
2,u108339,0.015010,0.072315,0.065400,0.061242,0.002470,0.006203,0.026775,0.002806,0.005460,...,0.009630,0.010440,0.0,0.117899,0.023279,0.247010,0.029281,0.065050,0.030370,0.013853


Датасет готов - можно попробовать обучить модель. Загрузим нашу разметку

In [43]:
target = pd.read_csv("./users_churn.csv")
target.head(3)

,uid,churn
0,u107120,0
1,u102277,0
2,u102444,0


In [44]:
X = pd.merge(user_embeddings, target, 'left')
X.head(3)

,uid,topic_0,topic_1,topic_2,topic_3,topic_4,topic_5,topic_6,topic_7,topic_8,...,topic_11,topic_12,topic_13,topic_14,topic_15,topic_16,topic_17,topic_18,topic_19,churn
0,u105138,0.030297,0.063619,0.083210,0.007537,0.000000,0.010949,0.031341,0.000000,0.042386,...,0.006087,0.0,0.088453,0.041973,0.247529,0.050127,0.037741,0.042514,0.040250,0
1,u108690,0.007978,0.161570,0.049986,0.038571,0.003282,0.010306,0.025159,0.000000,0.001911,...,0.011973,0.0,0.099249,0.123617,0.097997,0.021764,0.048871,0.017788,0.042099,1
2,u108339,0.015010,0.072315,0.065400,0.061242,0.002470,0.006203,0.026775,0.002806,0.005460,...,0.010440,0.0,0.117899,0.023279,0.247010,0.029281,0.065050,0.030370,0.013853,1


In [45]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

import matplotlib.pyplot as plt

In [46]:
# разделим данные на train/test
X_train, X_test, y_train, y_test = train_test_split(X[[f'topic_{i}' for i in range(N_topic)]], 
                                                    X['churn'], random_state=0)

In [47]:
model = LogisticRegression()
# обучим 
model.fit(X_train, y_train)

LogisticRegression()

In [48]:
# наши прогнозы для тестовой выборки
preds = model.predict_proba(X_test)[:, 1]
preds[:10]

array([0.0801101 , 0.04136444, 0.52324908, 0.14332266, 0.02701511,
       0.02448   , 0.11446716, 0.03830478, 0.18730664, 0.30667842])

In [49]:
metrics_df = pd.DataFrame(columns=['model', 'thresh', 'F-Score', 'Precision', 'Recall', 'ROC AUC'])
metrics_df

,model,thresh,F-Score,Precision,Recall,ROC AUC


In [50]:
from sklearn.metrics import (f1_score, roc_auc_score, precision_score,
                             classification_report, precision_recall_curve, confusion_matrix)

In [51]:
precision, recall, thresholds = precision_recall_curve(y_test, preds)
fscore = (2 * precision * recall) / (precision + recall)
# locate the index of the largest f score
ix = np.argmax(fscore)
print(f'Best Threshold={thresholds[ix]}, F-Score={fscore[ix]:.3f}, Precision={precision[ix]:.3f}, Recall={recall[ix]:.3f}')                                                                        

Best Threshold=0.28301770164390494, F-Score=0.695, Precision=0.670, Recall=0.722


In [52]:
roc_auc = roc_auc_score(y_test, preds)
roc_auc

0.9520134891563463

In [56]:
metrics_df = metrics_df.append({
    'model': FUNC.__name__,
    'thresh': thresholds[ix],
    'F-Score': fscore[ix],
    'Precision': precision[ix],
    'Recall': recall[ix],
    'ROC AUC': roc_auc
}, ignore_index=True)

metrics_df

C:\Users\Andrew\AppData\Local\Temp\ipykernel_5332\1288159249.py:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  metrics_df = metrics_df.append({


,model,thresh,F-Score,Precision,Recall,ROC AUC
0,mean,0.283018,0.695481,0.670455,0.722449,0.952013
1,mean,0.283018,0.695481,0.670455,0.722449,0.952013
2,mean,0.283018,0.695481,0.670455,0.722449,0.952013
3,mean,0.283018,0.695481,0.670455,0.722449,0.952013


2. Модифицировать код функции get_user_embedding таким образом, чтобы считалось не среднее (как в примере np.mean), а медиана. Применить такое преобразование к данным, обучить модель прогнозирования оттока и посчитать метрики качества и сохранить их: roc auc, precision/recall/f_score (для 3 последних - подобрать оптимальный порог)

In [57]:
FUNC = np.median

In [58]:
user_embeddings = pd.DataFrame([i for i in users['articles'].apply(lambda x: get_user_embedding(x, doc_dict, FUNC))])
user_embeddings.columns = [f'topic_{i}' for i in range(N_topic)]
user_embeddings['uid'] = users['uid'].values
user_embeddings = user_embeddings[['uid']+[f'topic_{i}' for i in range(N_topic)]]
user_embeddings.head(3)

,uid,topic_0,topic_1,topic_2,topic_3,topic_4,topic_5,topic_6,topic_7,topic_8,...,topic_10,topic_11,topic_12,topic_13,topic_14,topic_15,topic_16,topic_17,topic_18,topic_19
0,u105138,0.000000,0.028833,0.010634,0.000000,0.0,0.0,0.000000,0.0,0.0,...,0.011481,0.0,0.0,0.099551,0.033503,0.246052,0.047846,0.013464,0.000000,0.00000
1,u108690,0.000000,0.175896,0.047244,0.015629,0.0,0.0,0.027799,0.0,0.0,...,0.000000,0.0,0.0,0.052159,0.136046,0.099434,0.009979,0.028001,0.000000,0.03156
2,u108339,0.014827,0.076333,0.044861,0.059514,0.0,0.0,0.016530,0.0,0.0,...,0.000000,0.0,0.0,0.090211,0.000000,0.178681,0.016239,0.058480,0.013735,0.00000


In [59]:
X = pd.merge(user_embeddings, target, 'left')
X.head(3)

,uid,topic_0,topic_1,topic_2,topic_3,topic_4,topic_5,topic_6,topic_7,topic_8,...,topic_11,topic_12,topic_13,topic_14,topic_15,topic_16,topic_17,topic_18,topic_19,churn
0,u105138,0.000000,0.028833,0.010634,0.000000,0.0,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.099551,0.033503,0.246052,0.047846,0.013464,0.000000,0.00000,0
1,u108690,0.000000,0.175896,0.047244,0.015629,0.0,0.0,0.027799,0.0,0.0,...,0.0,0.0,0.052159,0.136046,0.099434,0.009979,0.028001,0.000000,0.03156,1
2,u108339,0.014827,0.076333,0.044861,0.059514,0.0,0.0,0.016530,0.0,0.0,...,0.0,0.0,0.090211,0.000000,0.178681,0.016239,0.058480,0.013735,0.00000,1


In [60]:
# разделим данные на train/test
X_train, X_test, y_train, y_test = train_test_split(X[[f'topic_{i}' for i in range(N_topic)]], 
                                                    X['churn'], random_state=0)

In [61]:
model = LogisticRegression()
# обучим 
model.fit(X_train, y_train)

LogisticRegression()

In [62]:
# наши прогнозы для тестовой выборки
preds = model.predict_proba(X_test)[:, 1]
preds[:10]

array([0.07892789, 0.02965685, 0.59254406, 0.21848318, 0.03490676,
       0.02452738, 0.08651015, 0.01834537, 0.2470185 , 0.51430849])

In [63]:
precision, recall, thresholds = precision_recall_curve(y_test, preds)
fscore = (2 * precision * recall) / (precision + recall)
# locate the index of the largest f score
ix = np.argmax(fscore)
print(f'Best Threshold={thresholds[ix]}, F-Score={fscore[ix]:.3f}, Precision={precision[ix]:.3f}, Recall={recall[ix]:.3f}')                                                                        

Best Threshold=0.25250277913100283, F-Score=0.713, Precision=0.636, Recall=0.812


In [64]:
roc_auc = roc_auc_score(y_test, preds)
roc_auc

0.954818303389732

In [65]:
metrics_df = metrics_df.append({
    'model': FUNC.__name__,
    'thresh': thresholds[ix],
    'F-Score': fscore[ix],
    'Precision': precision[ix],
    'Recall': recall[ix],
    'ROC AUC': roc_auc
}, ignore_index=True)

metrics_df

C:\Users\Andrew\AppData\Local\Temp\ipykernel_5332\1288159249.py:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  metrics_df = metrics_df.append({


,model,thresh,F-Score,Precision,Recall,ROC AUC
0,mean,0.283018,0.695481,0.670455,0.722449,0.952013
1,mean,0.283018,0.695481,0.670455,0.722449,0.952013
2,mean,0.283018,0.695481,0.670455,0.722449,0.952013
3,mean,0.283018,0.695481,0.670455,0.722449,0.952013
4,median,0.252503,0.713262,0.635783,0.812245,0.954818



3. Повторить п.2, но используя уже не медиану, а max


In [66]:
FUNC = np.max

In [67]:
user_embeddings = pd.DataFrame([i for i in users['articles'].apply(lambda x: get_user_embedding(x, doc_dict, FUNC))])
user_embeddings.columns = [f'topic_{i}' for i in range(N_topic)]
user_embeddings['uid'] = users['uid'].values
user_embeddings = user_embeddings[['uid']+[f'topic_{i}' for i in range(N_topic)]]
user_embeddings.head(3)

,uid,topic_0,topic_1,topic_2,topic_3,topic_4,topic_5,topic_6,topic_7,topic_8,...,topic_10,topic_11,topic_12,topic_13,topic_14,topic_15,topic_16,topic_17,topic_18,topic_19
0,u105138,0.096260,0.163515,0.382124,0.045223,0.000000,0.039937,0.109108,0.000000,0.223131,...,0.050446,0.036523,0.0,0.172286,0.138095,0.530587,0.112567,0.129990,0.156231,0.193904
1,u108690,0.030274,0.260131,0.092725,0.113406,0.019693,0.049323,0.056603,0.000000,0.011463,...,0.000000,0.042808,0.0,0.365259,0.246344,0.187805,0.085155,0.157226,0.055848,0.124268
2,u108339,0.026512,0.123614,0.178587,0.149621,0.014820,0.025445,0.068612,0.016835,0.032763,...,0.057783,0.062643,0.0,0.264470,0.116112,0.564141,0.071874,0.174894,0.098067,0.058666


In [68]:
X = pd.merge(user_embeddings, target, 'left')
X.head(3)

,uid,topic_0,topic_1,topic_2,topic_3,topic_4,topic_5,topic_6,topic_7,topic_8,...,topic_11,topic_12,topic_13,topic_14,topic_15,topic_16,topic_17,topic_18,topic_19,churn
0,u105138,0.096260,0.163515,0.382124,0.045223,0.000000,0.039937,0.109108,0.000000,0.223131,...,0.036523,0.0,0.172286,0.138095,0.530587,0.112567,0.129990,0.156231,0.193904,0
1,u108690,0.030274,0.260131,0.092725,0.113406,0.019693,0.049323,0.056603,0.000000,0.011463,...,0.042808,0.0,0.365259,0.246344,0.187805,0.085155,0.157226,0.055848,0.124268,1
2,u108339,0.026512,0.123614,0.178587,0.149621,0.014820,0.025445,0.068612,0.016835,0.032763,...,0.062643,0.0,0.264470,0.116112,0.564141,0.071874,0.174894,0.098067,0.058666,1


In [69]:
# разделим данные на train/test
X_train, X_test, y_train, y_test = train_test_split(X[[f'topic_{i}' for i in range(N_topic)]], 
                                                    X['churn'], random_state=0)

In [70]:
model = LogisticRegression()
# обучим 
model.fit(X_train, y_train)

LogisticRegression()

In [72]:
# наши прогнозы для тестовой выборки
preds = model.predict_proba(X_test)[:, 1]
preds[:10]

array([0.04044989, 0.00304959, 0.48778093, 0.06234194, 0.01745722,
       0.0011732 , 0.09279127, 0.02701511, 0.02352966, 0.3958556 ])

In [73]:
precision, recall, thresholds = precision_recall_curve(y_test, preds)
fscore = (2 * precision * recall) / (precision + recall)
# locate the index of the largest f score
ix = np.argmax(fscore)
print(f'Best Threshold={thresholds[ix]}, F-Score={fscore[ix]:.3f}, Precision={precision[ix]:.3f}, Recall={recall[ix]:.3f}')                                                                        

Best Threshold=0.3557174760519096, F-Score=0.712, Precision=0.723, Recall=0.702


In [74]:
roc_auc = roc_auc_score(y_test, preds)
roc_auc

0.9571765800337229

In [75]:
metrics_df = metrics_df.append({
    'model': FUNC.__name__,
    'thresh': thresholds[ix],
    'F-Score': fscore[ix],
    'Precision': precision[ix],
    'Recall': recall[ix],
    'ROC AUC': roc_auc
}, ignore_index=True)

metrics_df

C:\Users\Andrew\AppData\Local\Temp\ipykernel_5332\1288159249.py:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  metrics_df = metrics_df.append({


,model,thresh,F-Score,Precision,Recall,ROC AUC
0,mean,0.283018,0.695481,0.670455,0.722449,0.952013
1,mean,0.283018,0.695481,0.670455,0.722449,0.952013
2,mean,0.283018,0.695481,0.670455,0.722449,0.952013
3,mean,0.283018,0.695481,0.670455,0.722449,0.952013
4,median,0.252503,0.713262,0.635783,0.812245,0.954818
5,amax,0.355717,0.712215,0.722689,0.702041,0.957177


4. *Воспользовавшись полученными знаниями из п.1, повторить пункт 2, но уже взвешивая новости по tfidf (взяв список новостей пользователя)
	- подсказка 1: нужно получить веса-коэффициенты для каждого документа. Не все документы одинаково информативны и несут какой-то положительный сигнал
	- подсказка 2: нужен именно idf, как вес.

In [76]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [77]:
users['articles_str'] = users['articles'].apply(lambda x: x.replace('[','').replace(']', '').replace(',', ''))

users['articles_str'].iloc[0]

'293672 293328 293001 293622 293126 1852'

In [78]:
tfidf = TfidfVectorizer()
tfidf.fit(users['articles_str'])

TfidfVectorizer()

In [79]:
idf = pd.DataFrame({'article_id': tfidf.get_feature_names_out(),
                    'idf': tfidf.idf_})

idf

,article_id,idf
0,10,8.888710
1,100,7.907880
2,1000,8.041412
3,1001,8.888710
4,1002,8.888710
...,...,...
14776,995,8.377884
14777,996,8.195562
14778,997,8.601027
14779,998,9.294175


In [80]:
def get_user_embedding_idf(user_articles_list, doc_dict):
    user_articles_list = eval(user_articles_list)
    
    user_vector = np.zeros((len(user_articles_list), N_topic))
    for i, doc_id in enumerate(user_articles_list):
        try:
            weight = idf[idf['article_id'] == str(doc_id)]['idf'].values[0]
        except Exception as e:
            weight = 0
        user_vector[i] = doc_dict[doc_id] * weight

    user_vector = np.median(user_vector, axis=0)
    return user_vector

In [81]:
from tqdm import tqdm
tqdm.pandas()

user_embeddings = pd.DataFrame([i for i in users['articles'].progress_apply(lambda x: get_user_embedding_idf(x, doc_dict))])
user_embeddings.columns = [f'topic_{i}' for i in range(N_topic)]
user_embeddings['uid'] = users['uid'].values
user_embeddings = user_embeddings[['uid']+[f'topic_{i}' for i in range(N_topic)]]
user_embeddings.head(3)

100%|█████████████████████████████████████| 8000/8000 [01:17<00:00, 102.99it/s]


,uid,topic_0,topic_1,topic_2,topic_3,topic_4,topic_5,topic_6,topic_7,topic_8,...,topic_10,topic_11,topic_12,topic_13,topic_14,topic_15,topic_16,topic_17,topic_18,topic_19
0,u105138,0.000000,0.247991,0.098830,0.000000,0.0,0.0,0.000000,0.0,0.0,...,0.09409,0.0,0.0,0.855766,0.300597,2.116304,0.394325,0.115801,0.00000,0.000000
1,u108690,0.000000,1.493066,0.419938,0.138925,0.0,0.0,0.241489,0.0,0.0,...,0.00000,0.0,0.0,0.473331,1.243041,0.792859,0.092747,0.248893,0.00000,0.245857
2,u108339,0.123073,0.615931,0.364422,0.498598,0.0,0.0,0.137451,0.0,0.0,...,0.00000,0.0,0.0,0.795493,0.000000,1.457755,0.137340,0.484627,0.11507,0.000000


In [82]:
X = pd.merge(user_embeddings, target, 'left')
X.head(3)

,uid,topic_0,topic_1,topic_2,topic_3,topic_4,topic_5,topic_6,topic_7,topic_8,...,topic_11,topic_12,topic_13,topic_14,topic_15,topic_16,topic_17,topic_18,topic_19,churn
0,u105138,0.000000,0.247991,0.098830,0.000000,0.0,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.855766,0.300597,2.116304,0.394325,0.115801,0.00000,0.000000,0
1,u108690,0.000000,1.493066,0.419938,0.138925,0.0,0.0,0.241489,0.0,0.0,...,0.0,0.0,0.473331,1.243041,0.792859,0.092747,0.248893,0.00000,0.245857,1
2,u108339,0.123073,0.615931,0.364422,0.498598,0.0,0.0,0.137451,0.0,0.0,...,0.0,0.0,0.795493,0.000000,1.457755,0.137340,0.484627,0.11507,0.000000,1


In [83]:
# разделим данные на train/test
X_train, X_test, y_train, y_test = train_test_split(X[[f'topic_{i}' for i in range(N_topic)]], 
                                                    X['churn'], random_state=0)

In [84]:
model = LogisticRegression()
# обучим 
model.fit(X_train, y_train)

LogisticRegression()

In [85]:
# наши прогнозы для тестовой выборки
preds = model.predict_proba(X_test)[:, 1]
preds[:10]

array([4.52960167e-02, 1.38617370e-04, 8.41949976e-01, 1.60373902e-01,
       2.61150643e-03, 9.25785468e-04, 2.99756682e-02, 7.00582772e-06,
       1.30176460e-01, 8.64955270e-01])

In [86]:
precision, recall, thresholds = precision_recall_curve(y_test, preds)
fscore = (2 * precision * recall) / (precision + recall)
# locate the index of the largest f score
ix = np.argmax(fscore)
print(f'Best Threshold={thresholds[ix]}, F-Score={fscore[ix]:.3f}, Precision={precision[ix]:.3f}, Recall={recall[ix]:.3f}')                                                                        

Best Threshold=0.24976158517024835, F-Score=0.782, Precision=0.691, Recall=0.902


In [87]:
roc_auc = roc_auc_score(y_test, preds)
roc_auc

0.9742054770626198

In [90]:
metrics_df = metrics_df.append({
    'model': 'idf_median',
    'thresh': thresholds[ix],
    'F-Score': fscore[ix],
    'Precision': precision[ix],
    'Recall': recall[ix],
    'ROC AUC': roc_auc
}, ignore_index=True)

metrics_df

C:\Users\Andrew\AppData\Local\Temp\ipykernel_5332\1604886773.py:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  metrics_df = metrics_df.append({


,model,thresh,F-Score,Precision,Recall,ROC AUC
0,mean,0.283018,0.695481,0.670455,0.722449,0.952013
1,mean,0.283018,0.695481,0.670455,0.722449,0.952013
2,mean,0.283018,0.695481,0.670455,0.722449,0.952013
3,mean,0.283018,0.695481,0.670455,0.722449,0.952013
4,median,0.252503,0.713262,0.635783,0.812245,0.954818
5,amax,0.355717,0.712215,0.722689,0.702041,0.957177
6,idf_median,0.249762,0.782301,0.690625,0.902041,0.974205
7,idf_median,0.249762,0.782301,0.690625,0.902041,0.974205
8,idf_median,0.249762,0.782301,0.690625,0.902041,0.974205



5. Сформировать на выходе единую таблицу, сравнивающую качество 2/3 разных метода получения эмбедингов пользователей: median, max, idf_mean по метрикам roc_auc, precision, recall, f_score


In [91]:
metrics_df

,model,thresh,F-Score,Precision,Recall,ROC AUC
0,mean,0.283018,0.695481,0.670455,0.722449,0.952013
1,mean,0.283018,0.695481,0.670455,0.722449,0.952013
2,mean,0.283018,0.695481,0.670455,0.722449,0.952013
3,mean,0.283018,0.695481,0.670455,0.722449,0.952013
4,median,0.252503,0.713262,0.635783,0.812245,0.954818
5,amax,0.355717,0.712215,0.722689,0.702041,0.957177
6,idf_median,0.249762,0.782301,0.690625,0.902041,0.974205
7,idf_median,0.249762,0.782301,0.690625,0.902041,0.974205
8,idf_median,0.249762,0.782301,0.690625,0.902041,0.974205


6. Сделать самостоятельные выводы и предположения о том, почему тот или ной способ оказался эффективнее остальных